# MNIST Handwriten Digit Classification 

This is the main training file of the logistic/softmax classifier on MNIST.

#### Import All Relevant Modules

In [1]:
import os
import time as timer

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.transforms as T

## My own modules
import viz_utils as vu
import train_utils as tu
import model_utils as mu
###

import imp
imp.reload(mu)
imp.reload(tu)

/var/folders/wb/ttr9x7pn27q1_rp5vtjfr0s00000gn/T/ipykernel_18109/4166587151.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'train_utils' from '/Users/mghifary/Work/govtech/codes/AI/IF5171/deeplearning/src/train_utils.py'>

#### Constants

In [2]:
DATA_DIR = "data"
MODEL_DIR = "models"

EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
## Create the directory to store the model if not exists
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

AttributeError: module 'posixpath' has no attribute 'makedirs'

#### Load MNIST dataset

In [ ]:
# Load dataset
training_data = datasets.MNIST(
    root=DATA_DIR, 
    train=True, 
    download=True,
    transform=T.ToTensor(), #convert to Tensor and normalize to (0, 1)
)

test_data = datasets.MNIST(
    root=DATA_DIR,
    train=False,
    download=True,
    transform=T.ToTensor(), #convert to Tensor and normalize to (0, 1)
)

In [ ]:
# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
grid = vu.set_grid(train_dataloader.dataset.data, num_cells=56)
vu.show(grid)

In [ ]:
grid = vu.set_grid(test_dataloader.dataset.data, num_cells=56)
vu.show(grid)

#### Create the model

In [ ]:
print(f"Using {DEVICE} device")

# Define model
model = mu.SoftmaxClassifier(d_in=28*28, d_out=10).to(DEVICE)
print(model)

# Persistent file to store the model
model_path = os.path.join(MODEL_DIR, "softmax_mnist.pth")

In [ ]:
# print(f"Using {DEVICE} device")

# # Define model
# model = mu.MLP(d_in=28*28, d_out=10)
# model = model.to(DEVICE)
# print(model)

# # Persistent file to store the model
# model_path = os.path.join(MODEL_DIR, "mlp_mnist.pth")

In [ ]:
# print(f"Using {DEVICE} device")

# # Define model
# model = mu.ConvNet(d_out=10).to(DEVICE)
# print(model)

# # Persistent file to store the model
# model_path = os.path.join(MODEL_DIR, "convnet_mnist_v3.pth")

#### Define the loss function and the optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(
    model.parameters(), 
    lr=LEARNING_RATE
)

# optimizer = optim.Adam(
#     model.parameters(), 
#     lr=LEARNING_RATE
# )

#### Execute the training!

In [ ]:
for t in range(EPOCHS):
    print(f"Epoch {t+1} out of {EPOCHS}\n ------------")
    
    start = timer.time()
    tu.train(train_dataloader, model, loss_fn, optimizer)
    elapsed_time = timer.time() - start # this timing method ONLY works for CPU computation, not for GPU/cuda calls
    print(f" > Training time: {elapsed_time:>.2f} seconds")
    
    tu.test(test_dataloader, model, loss_fn)
    
    # Save model
    torch.save(model.state_dict(), model_path)
    print(f"Model {model_path} stored!")
    
print("Done!")